In [ ]:
import os
import cv2

In [ ]:
def split_video(input_path, output_dir, max_duration_sec=30):
    current_part = 1
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / fps

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    frames_per_split = max_duration_sec * fps
    frame_count = 0

    out = None

    base_filename = os.path.splitext(os.path.basename(input_path))[0]

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frames_per_split == 0:
            if out:
                out.release()
            output_filename = os.path.join(
                output_dir,
                f"{base_filename}_part_{current_part:03d}.mp4"
            )
            out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))
            current_part += 1

        out.write(frame)
        frame_count += 1

    cap.release()
    if out:
        out.release()

    print(f"Video split into {current_part - 1} parts of max {max_duration_sec} seconds each.")



In [ ]:
# split one video into chunks of 30 sec
video_path = "./0_true_positive/4th_may/2025-05-04_13-56-04-549523_clip_990.mp4"
output_path = "./test"
split_video(video_path, output_path)

In [ ]:
# split all videos in root_dir into chunks of 30 sec
root_dir = "./1_cash_vs_items"
output_path = "./1_cash_vs_items/trim_videos"
# Video file extensions to look for
video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.webm')
for dirpath, _, filenames in os.walk(root_dir):
    for file in filenames:
        if file.lower().endswith(video_extensions):
            video_path = os.path.join(dirpath, file)
            split_video(video_path, output_path)